# TRABALHANDO COM ARQUIVOS(ENTRADA E SAIDA) PARTE II

## ARQUIVOS DE DADOS JLD

Arquivo JDL é um formato amplamente utilizado para armazenamento de dados com a linguagem de programação Julia. JLD é um formato derivado do HDF5 aplicado para o armazenamento de dados científicos de grande volume. 

Arquivos HDF5 são organizados em uma estrutura hierárquica. Esta, pode ser vista como uma árvore de diretórios dentro de um arquivo. Suas duas estruturas primárias são groups e datasets (HDF Group, 2007). Um HDF5 group é uma estrutura de agrupamento que pode conter zero ou mais instâncias de grupos ou datasets, juntamente com seus metadados, enquanto um HDF5 dataset é um array multidimensional de dados também em conjunto com seus metadados.A figura abaixo representa a estrutura de um arquivo HDF5 [(PERANTUNES,2007)](https://projetos.inf.ufsc.br/arquivos_projetos/projeto_733/hilton_perantunes_ine5631.pdf). 
![Estrutura do HDF5](Figuras/hdf5.png)
Um grupo contém zero ou mais HDF5 objects (grupos ou datasets), e é composto por duas partes: a primeira é formada por um cabeçalho contendo o nome do grupo e uma lista de atributos, sendo que a segunda parte é uma tabela de símbolos com uma listagem dos objetos HDF5 que pertencem ao grupo . Já um dataset possui um cabeçalho e um array de dados  [(PERANTUNES,2007)](https://projetos.inf.ufsc.br/arquivos_projetos/projeto_733/hilton_perantunes_ine5631.pdf).

Em comparação com HDF5, arquivos JLD adicionam automaticamente atributos e convenções de nomenclatura para preservar as informações de tipo para cada objeto. Vale resaltar que arquivos * .jld são arquivos HDF5 com recursos extras e, consequentemente, pode ser lido por qualquer linguagem que suporte HDF5. Para mais detalhes consulte [JuliaIO/JLD](https://github.com/JuliaIO/JLD.jl/) . Uma grande vantagem é que a saída do arquivo é do tipo dicionário e portante vale todas as operações para este tipo no notebook "TIPOS DE DADOS". Para instalar o pacote JLD use:
```julia
Pkg.add("JLD")
```

**Criar,  Abrir e gravar dados no arquivo JLD**

Para criar um novo arquivo basta abrir ou escrever um arquivo que não existe ainda e `Julia` irá escrever no disco o arquivo. Para escrever em um arquivo novo use:
```julia
save("nome_arquivo.jld", "key_1", valor_dado, "key_2", valor_dado,...)
```

In [11]:
using JLD

#### MODO BÁSICO

Neste primeiro contato, faremos apenas uma introdução rápida sobre a manipulação dos arquivos no formato `JLD` de tal forma que seja suficiente para o usuário gravar informações no disco e recuperá-las para trabalhar com os dados. A desvantagem é que alterado os dados de uma `key` é necessário salva todos os dados novamente no arquivo em disco.

In [12]:
# dados
matriz = rand(Float64,6,6)
vetor = rand(5)
nome = "Arquivos"

"Arquivos"

In [13]:
# salvar arquivo em disco 
save("arquivo_basico.jld", "matriz", matriz, "vetor", vetor , "nome", nome)

In [14]:
# ler os dados do arquivo gravado em disco
arquivo_basico = load("arquivo_basico.jld")

Dict{String,Any} with 3 entries:
  "matriz" => [0.660187 0.786183 … 0.240348 0.261368; 0.0133873 0.970472 … 0.22…
  "vetor"  => [0.600473, 0.158084, 0.414937, 0.674712, 0.269395]
  "nome"   => "Arquivos"

In [15]:
# deletar uma key e seus valores
delete!(arquivo_basico, "vetor")

Dict{String,Any} with 2 entries:
  "matriz" => [0.660187 0.786183 … 0.240348 0.261368; 0.0133873 0.970472 … 0.22…
  "nome"   => "Arquivos"

In [16]:
# listar as keys
keys(arquivo_basico)

Base.KeySet for a Dict{String,Any} with 2 entries. Keys:
  "matriz"
  "nome"

In [17]:
# listar os valores das keys
values(arquivo_basico)

Base.ValueIterator for a Dict{String,Any} with 2 entries. Values:
  [0.660187 0.786183 … 0.240348 0.261368; 0.0133873 0.970472 … 0.224216 0.04570…
  "Arquivos"

In [18]:
# mostrar os valores da key "matriz"
arquivo_basico["matriz"]

6×6 Array{Float64,2}:
 0.660187   0.786183  0.479609   0.383642   0.240348  0.261368 
 0.0133873  0.970472  0.0267761  0.0334408  0.224216  0.0457048
 0.776844   0.376885  0.854873   0.662792   0.137693  0.223808 
 0.966029   0.411159  0.229009   0.200476   0.564937  0.877581 
 0.807123   0.750271  0.0288055  0.36686    0.16131   0.279597 
 0.248982   0.985183  0.827886   0.222823   0.817785  0.727108 

In [19]:
# altera matriz e nome
arquivo_basico["matriz"] = rand(3,2)
arquivo_basico["nome"]  =  "Testando";

In [20]:
# salvar alterações em disco
save("arquivo_basico.jld", arquivo_basico)

In [21]:
# ler os dados do arquivo gravado em disco. Veja que a matriz mudou.
load("arquivo_basico.jld")

Dict{String,Any} with 2 entries:
  "matriz" => [0.0519831 0.965207; 0.691028 0.0453523; 0.795819 0.917402]
  "nome"   => "Testando"

#### MODO AVANÇADO

Nesta parte será trabalhado a abertura do arquivo para manipular os dados e fechamendo após as operações. Isto é importante pois permite salvar apenas as `keys` alteradas.

Um novo arquivo será criado quando executado o comando `jldopen` com atributo de gravação **"w"** e vinculado a uma variável. Sintaxe:
```julia
var_dados = jldopen("nome_arquivo.jld", atributo)

```
`Tabela de atributos`

<table border="1" class="docutils" style="float: left;">
<colgroup>
<col width="11%" />
<col width="89%" />
</colgroup>
<tbody valign="top">
<tr class="row-odd"><td>"r"</td>
<td>Somente ler os dados</td>
</tr>
<tr class="row-even"><td>"r+"</td>
<td>Ler e escrever mantendo os dados já escritos</td>
</tr>
<tr class="row-odd"><td>"w"</td>
<td>Escrever destruindo todos os dados existentes. CUIDADO! RECOMENDÁVEL SOMENTE PARA ARQUIVOS NOVOS</td>
</tr>

</tbody>
</table>

In [22]:
using JLD

In [23]:
arquivo_avancado = jldopen("arquivo_avancado.jld", "w")

Julia data file version 0.1.2: arquivo_avancado.jld

Uma vez aberto o arquivo não é possível abri-lo novamente com outro atributo, se tentar ocorrerá este erro:

        HDF5-DIAG: Error detected in HDF5 (1.8.16) thread 139655273372672:
          #000: ../../../src/H5F.c line 522 in H5Fcreate(): unable to create file
            major: File accessibilty
            minor: Unable to open file
          #001: ../../../src/H5Fint.c line 1024 in H5F_open(): unable to truncate a file which is already open
            major: File accessibilty
            minor: Unable to open file
            
Este erro informa que o arquivo está inacessível e que ja está aberto. Para alterar o atributo e abrir o arquivo novamente é necessário fecha-lo primeiro. Muitas vezes não sabemos qual objeto está vinculado ao arquivo aberto, dessa forma utilize a função `whos()` basta procurar pelo objeto do tipo `JLD.JldFile` e fecha-lo.

In [24]:
matriz = rand(Float64,6,6) # matriz aleatoria do tipo float64
vetor  = rand(5)           # vetor aleatóro do tipo float64
nome   = "Arquivos"        # string 
funcao = "x*sind(x)"       # função do tipo string

"x*sind(x)"

In [25]:
# gravando dados no arquivo
write(arquivo_avancado, "Matriz", matriz)
write(arquivo_avancado, "vetor" , vetor)
write(arquivo_avancado, "nome" , nome)
write(arquivo_avancado, "função" , funcao)

In [26]:
# fechando o arquivo
close(arquivo_avancado)

** Lendo arquivo de dados**

Para exibir os dados de uma `key` use:
```julia
read(nome_var_arquivo["nome_key"])
```
Sendo "nome_var_arquivo" o nome da variável que é atribído o arquivo gravado em disco e "nome_key" o nome da chave.

Para acessar valores específicos da `key` use:
```julia
read(nome_var_arquivo["nome_key"])[ordem_dado]
```
`ordem_dado` é a referência ao registro do dado num vetor, matriz e outros tipos básicos

In [27]:
using JLD

In [28]:
# vinculando a "arquivo_avancado" ao arquivo gravado em disco
arquivo_avancado = jldopen("arquivo_avancado.jld", "r+")

Julia data file version 0.1.2: arquivo_avancado.jld

In [29]:
# verificar o tipo
typeof(arquivo_avancado)

JLD.JldFile

In [30]:
# ler os dados de arquivo_dados
read(arquivo_avancado)

Dict{String,Any} with 4 entries:
  "função" => "x*sind(x)"
  "vetor"  => [0.23393, 0.141766, 0.645631, 0.232812, 0.323041]
  "nome"   => "Arquivos"
  "Matriz" => [0.935366 0.81458 … 0.61642 0.0203982; 0.794274 0.710353 … 0.0343…

In [31]:
# deletar uma key e seus valores
delete!(arquivo_avancado, "vetor")

false

In [32]:
# ler as keys. key "vetor" foi deletada
keys(read(arquivo_avancado))

Base.KeySet for a Dict{String,Any} with 3 entries. Keys:
  "função"
  "nome"
  "Matriz"

In [33]:
values(read(arquivo_avancado))

Base.ValueIterator for a Dict{String,Any} with 3 entries. Values:
  "x*sind(x)"
  "Arquivos"
  [0.935366 0.81458 … 0.61642 0.0203982; 0.794274 0.710353 … 0.0343351 0.179585…

In [34]:
# Listar todos os dados do registro "1"
read(arquivo_avancado["Matriz"])

6×6 Array{Float64,2}:
 0.935366   0.81458   0.437231  0.690178   0.61642    0.0203982
 0.794274   0.710353  0.266162  0.973579   0.0343351  0.179585 
 0.545826   0.594448  0.487492  0.833393   0.342145   0.267891 
 0.0414543  0.921694  0.135236  0.653629   0.324803   0.580684 
 0.387451   0.780827  0.87902   0.0871821  0.249427   0.188895 
 0.614234   0.988025  0.758377  0.167114   0.398239   0.623417 

In [35]:
# listar somente os elementos da segunda linha
read(arquivo_avancado["Matriz"])[2,:]

6-element Array{Float64,1}:
 0.7942739477990566  
 0.7103527444875375  
 0.2661621856007501  
 0.973579302813331   
 0.034335064915465896
 0.17958547680744896 

In [36]:
# alterar o valor de um elemento
arquivo_avancado["Matriz"][1,1] = 9999

9999

In [37]:
read(arquivo_avancado["Matriz"])

6×6 Array{Float64,2}:
 9999.0        0.81458   0.437231  0.690178   0.61642    0.0203982
    0.794274   0.710353  0.266162  0.973579   0.0343351  0.179585 
    0.545826   0.594448  0.487492  0.833393   0.342145   0.267891 
    0.0414543  0.921694  0.135236  0.653629   0.324803   0.580684 
    0.387451   0.780827  0.87902   0.0871821  0.249427   0.188895 
    0.614234   0.988025  0.758377  0.167114   0.398239   0.623417 

In [38]:
# Listar todos os dados do registro "3"
read(arquivo_avancado["nome"])

"Arquivos"

In [39]:
# transformando a string da key 4 em função
@eval f(x) = $(Meta.parse(read(arquivo_avancado["função"])))

f (generic function with 1 method)

In [40]:
f(30)

15.0

In [41]:
# definindo um tipo específico

mutable struct Amostra_obj
     material  ::String
     massa     ::Float64
     raio      ::Float64 
     densidade ::Float64    
end

In [42]:
objeto1 = Amostra_obj("isopor", 0.001, 0.1,  0.0001)

Amostra_obj("isopor", 0.001, 0.1, 0.0001)

In [43]:
# salvar no arquivo
write(arquivo_avancado, "objeto1" , objeto1 )

In [44]:
# atribuir objeto1 à key "objeto1" 
amostra_1 = read(arquivo_avancado["objeto1"])

# listar as caracteristicas do objeto amostra_1
amostra_1.material , amostra_1.massa , amostra_1.raio , amostra_1.densidade 

("isopor", 0.001, 0.1, 0.0001)

É possível gravar em uma `key` um tipo específico. Isto é útil para ter sempre disponível um tipo específico.

In [46]:
# salvar no arquivo
write(arquivo_avancado, "tipo_amostra" , Amostra_obj )

In [47]:
amostra_metal = read(arquivo_avancado["tipo_amostra"])("metal", 0.1, 0.1,  0.15)

Amostra_obj("metal", 0.1, 0.1, 0.15)

In [48]:
# gravando o tipo no arquivo
write(arquivo_avancado, "amostra_metal" , amostra_metal )

In [49]:
# observe os valores alterados dos elementos e a ausência da key "vetor"
read(arquivo_avancado)

Dict{String,Any} with 6 entries:
  "amostra_metal" => Amostra_obj("metal", 0.1, 0.1, 0.15)
  "função"        => "x*sind(x)"
  "tipo_amostra"  => Amostra_obj
  "objeto1"       => Amostra_obj("isopor", 0.001, 0.1, 0.0001)
  "nome"          => "Arquivos"
  "Matriz"        => [9999.0 0.81458 … 0.61642 0.0203982; 0.794274 0.710353 … 0…

In [50]:
# fechar arquivo
close(arquivo_avancado)

In [51]:
# depois de fechado, os dados não podem ser lidos a partir da variável vinculada
read(arquivo_avancado)

ErrorException: File or object has been closed

**Ler e alterar dados em um arquivo jld sem atribuição de variável**

Para somente ler o conteúdo de um arquivo sem ter a necessidade de fecha-lo depois, use o comando: 
```julia 
read(jldopen("nome_arquivo.jld", "atributo"))
```
Para ser prático, use o atributo "r+" pois assim é possível alterar valores dos elementos de contidos em uma `key`.

Para gravar uma nova `key` use:
```julia
write(jldopen("nome_arquivo.jld", "r+"), "nova_key","valor_dado")
```
Não use ```write("nome_arquivo.jld","key","Valor_Dados") ``` para gravar diretamente, todos os dados anteriores serão perdidos.

In [52]:
# use o atributo "r+" para ler e alterar dados do arquivo.
read(jldopen("arquivo_avancado.jld", "r+"))

Dict{String,Any} with 6 entries:
  "amostra_metal" => Amostra_obj("metal", 0.1, 0.1, 0.15)
  "função"        => "x*sind(x)"
  "tipo_amostra"  => Amostra_obj
  "objeto1"       => Amostra_obj("isopor", 0.001, 0.1, 0.0001)
  "nome"          => "Arquivos"
  "Matriz"        => [9999.0 0.81458 … 0.61642 0.0203982; 0.794274 0.710353 … 0…

In [53]:
read(jldopen("arquivo_avancado.jld", "r+")["Matriz"])

6×6 Array{Float64,2}:
 9999.0        0.81458   0.437231  0.690178   0.61642    0.0203982
    0.794274   0.710353  0.266162  0.973579   0.0343351  0.179585 
    0.545826   0.594448  0.487492  0.833393   0.342145   0.267891 
    0.0414543  0.921694  0.135236  0.653629   0.324803   0.580684 
    0.387451   0.780827  0.87902   0.0871821  0.249427   0.188895 
    0.614234   0.988025  0.758377  0.167114   0.398239   0.623417 

In [54]:
read(jldopen("arquivo_avancado.jld", "r+")["Matriz"])[1,1]

9999.0

In [55]:
# Alterar o elemento [1,1] da matriz da key "1"
(jldopen("arquivo_avancado.jld", "r+")["Matriz"])[1,1] = 0000.0

ErrorException: File or object has been closed

In [56]:
# testar a mudança
read(jldopen("arquivo_avancado.jld", "r+"))

Dict{String,Any} with 6 entries:
  "amostra_metal" => Amostra_obj("metal", 0.1, 0.1, 0.15)
  "função"        => "x*sind(x)"
  "tipo_amostra"  => Amostra_obj
  "objeto1"       => Amostra_obj("isopor", 0.001, 0.1, 0.0001)
  "nome"          => "Arquivos"
  "Matriz"        => [9999.0 0.81458 … 0.61642 0.0203982; 0.794274 0.710353 … 0…

In [57]:
# gravar uma nova key ao arquivo
write(jldopen("arquivo_avancado.jld", "r+"), "Key_testes","so testes")

In [58]:
# remover uma key e seus valores
delete!(jldopen("arquivo_avancado.jld", "r+"), "nome")

false

In [59]:
# verificar a nova key e key "nome" deletada
read(jldopen("arquivo_avancado.jld", "r+"))

Dict{String,Any} with 6 entries:
  "amostra_metal" => Amostra_obj("metal", 0.1, 0.1, 0.15)
  "função"        => "x*sind(x)"
  "tipo_amostra"  => Amostra_obj
  "objeto1"       => Amostra_obj("isopor", 0.001, 0.1, 0.0001)
  "Key_testes"    => "so testes"
  "Matriz"        => [9999.0 0.81458 … 0.61642 0.0203982; 0.794274 0.710353 … 0…

**Filtragem de dados**



A função `filter()` permite filtrar. Sintaxe:
```julia
filter((key, valor) -> key == "nome_key", read(nome_arquivo_dados)) # filtrar key
filter((valor) -> valor operador_logico valor , read(nome_arquivo_dados)) # filtrar dados em uma key
```
List Comprehension

Sintaxe:
```julia
read(nome_arquivo_dados)["nome_key"][read(nome_arquivo_dados)["nome_key"].> valor]
```

In [60]:
using JLD

In [61]:
# ler os dados do arquivo e atribuir à variável dados_arquivo1
dados_arquivo1 = load("arquivo_avancado.jld")

Dict{String,Any} with 6 entries:
  "amostra_metal" => Amostra_obj("metal", 0.1, 0.1, 0.15)
  "função"        => "x*sind(x)"
  "tipo_amostra"  => Amostra_obj
  "objeto1"       => Amostra_obj("isopor", 0.001, 0.1, 0.0001)
  "Key_testes"    => "so testes"
  "Matriz"        => [9999.0 0.81458 … 0.61642 0.0203982; 0.794274 0.710353 … 0…

In [62]:
dados_arquivo1["Matriz"]

6×6 Array{Float64,2}:
 9999.0        0.81458   0.437231  0.690178   0.61642    0.0203982
    0.794274   0.710353  0.266162  0.973579   0.0343351  0.179585 
    0.545826   0.594448  0.487492  0.833393   0.342145   0.267891 
    0.0414543  0.921694  0.135236  0.653629   0.324803   0.580684 
    0.387451   0.780827  0.87902   0.0871821  0.249427   0.188895 
    0.614234   0.988025  0.758377  0.167114   0.398239   0.623417 

In [63]:
dados_arquivo1["Matriz"][dados_arquivo1["Matriz"] .> 0.9]

4-element Array{Float64,1}:
 9999.0               
    0.9216942974309947
    0.9880253337928839
    0.973579302813331 

In [64]:
# Este caso é melhor que load pois permite alterar o conteudo caso seja necessario
dados_arquivo2 = jldopen("arquivo_avancado.jld", "r+") 

Julia data file version 0.1.2: arquivo_avancado.jld

In [65]:
read(dados_arquivo2)

Dict{String,Any} with 6 entries:
  "amostra_metal" => Amostra_obj("metal", 0.1, 0.1, 0.15)
  "função"        => "x*sind(x)"
  "tipo_amostra"  => Amostra_obj
  "objeto1"       => Amostra_obj("isopor", 0.001, 0.1, 0.0001)
  "Key_testes"    => "so testes"
  "Matriz"        => [9999.0 0.81458 … 0.61642 0.0203982; 0.794274 0.710353 … 0…

In [66]:
filter((key, valor) -> key == "Matriz", read(dados_arquivo2))

┌ Warning: In `filter(f, dict)`, `f` is now passed a single pair instead of two arguments.
│   caller = top-level scope at In[66]:1
└ @ Core In[66]:1


Dict{String,Any} with 1 entry:
  "Matriz" => [9999.0 0.81458 … 0.61642 0.0203982; 0.794274 0.710353 … 0.034335…

In [67]:
filter((valor) -> valor > 0.9, read(dados_arquivo2)["Matriz"])

4-element Array{Float64,1}:
 9999.0               
    0.9216942974309947
    0.9880253337928839
    0.973579302813331 

In [68]:
# filtrar valores da matriz maiores que 0.8 e menores que 0.9
filter((valor) -> (valor > 0.8) & (valor < 0.9) , read(dados_arquivo2)["Matriz"])

3-element Array{Float64,1}:
 0.8145798156268091
 0.8790200614285186
 0.8333929406691067

In [69]:
read(dados_arquivo2)["Matriz"][read(dados_arquivo2)["Matriz"].> 0.9]

4-element Array{Float64,1}:
 9999.0               
    0.9216942974309947
    0.9880253337928839
    0.973579302813331 

In [72]:
# filtrar valores da matriz maiores que 0.8 e menores que 0.9
read(dados_arquivo2)["Matriz"][(read(dados_arquivo2)["Matriz"].> 0.8) & (read(dados_arquivo2)["Matriz"].< 0.9)] 

MethodError: MethodError: no method matching &(::BitArray{2}, ::BitArray{2})
Closest candidates are:
  &(::Any, ::Any, !Matched::Any, !Matched::Any...) at operators.jl:502

**Comparação entre Keys**

O comandos `setdiff` retorna a diferença entre os dois arquivos na forma de um dicionáro que contém os elementos que se encontram no primeiro arquivo mas não no segundo -  a ordem dos arquivos é importente. O comando `intersect` retorna um diconário que contém todos os elementos que pertence a ambos os arquivos.

In [73]:
# criação de um novo arquivo
save("arquivo_avancado_novo.jld", "1",50.05, "2", [1,2,3], "Key_testes", "so testes")
dados_arquivo_novo = load("arquivo_avancado_novo.jld")

Dict{String,Any} with 3 entries:
  "1"          => 50.05
  "2"          => [1, 2, 3]
  "Key_testes" => "so testes"

In [74]:
dados_arquivo2 = load("arquivo_avancado.jld")

Dict{String,Any} with 6 entries:
  "amostra_metal" => Amostra_obj("metal", 0.1, 0.1, 0.15)
  "função"        => "x*sind(x)"
  "tipo_amostra"  => Amostra_obj
  "objeto1"       => Amostra_obj("isopor", 0.001, 0.1, 0.0001)
  "Key_testes"    => "so testes"
  "Matriz"        => [9999.0 0.81458 … 0.61642 0.0203982; 0.794274 0.710353 … 0…

In [75]:
# listar as keys diferentes entre o dados_arquivo2 e dados_arquivo_novo
setdiff(dados_arquivo2, dados_arquivo_novo)

5-element Array{Pair{String,Any},1}:
 "amostra_metal" => Amostra_obj("metal", 0.1, 0.1, 0.15)                                                                                                                          
        "função" => "x*sind(x)"                                                                                                                                                   
  "tipo_amostra" => Amostra_obj                                                                                                                                                   
       "objeto1" => Amostra_obj("isopor", 0.001, 0.1, 0.0001)                                                                                                                     
        "Matriz" => [9999.0 0.81458 … 0.61642 0.0203982; 0.794274 0.710353 … 0.0343351 0.179585; … ; 0.387451 0.780827 … 0.249427 0.188895; 0.614234 0.988025 … 0.398239 0.623417]

In [76]:
# listar as keys diferentes entre dados_arquivo_novo e dados_arquivo2
setdiff(dados_arquivo_novo, dados_arquivo2)

2-element Array{Pair{String,Any},1}:
 "1" => 50.05    
 "2" => [1, 2, 3]

In [77]:
# elementos em comum entre os arquivos.
intersect(dados_arquivo2, dados_arquivo_novo)

1-element Array{Pair{String,Any},1}:
 "Key_testes" => "so testes"

**Map, MapReduce, Sum, Prod e List Comprehensions**

In [78]:
dados_arquivo3 = load("arquivo_avancado.jld")

Dict{String,Any} with 6 entries:
  "amostra_metal" => Amostra_obj("metal", 0.1, 0.1, 0.15)
  "função"        => "x*sind(x)"
  "tipo_amostra"  => Amostra_obj
  "objeto1"       => Amostra_obj("isopor", 0.001, 0.1, 0.0001)
  "Key_testes"    => "so testes"
  "Matriz"        => [9999.0 0.81458 … 0.61642 0.0203982; 0.794274 0.710353 … 0…

In [79]:
# mapeamento de uma função anonima aplicado a matriz do arquivo
map(x -> x*sin(x) , dados_arquivo3["Matriz"])

6×6 Array{Float64,2}:
 6360.23        0.592556  0.185137   0.439419   0.356364    0.000416059
    0.566599    0.463222  0.0700088  0.805056   0.00117867  0.0320779  
    0.283351    0.332922  0.228347   0.616892   0.114793    0.0709104  
    0.00171797  0.734246  0.0182332  0.397453   0.103652    0.318561   
    0.14639     0.549599  0.676946   0.0075911  0.0615707   0.0354695  
    0.354003    0.824943  0.521569   0.0277972  0.154435    0.363959   

In [80]:
mapreduce(x -> x*sin(x) , + , dados_arquivo3["Matriz"])

6370.69086181373

In [81]:
sum(dados_arquivo3["Matriz"])

10016.387999535571

In [82]:
prod(dados_arquivo3["Matriz"])

4.472812894051676e-12

In [83]:
[x*sin(x) for x = dados_arquivo3["Matriz"]]

6×6 Array{Float64,2}:
 6360.23        0.592556  0.185137   0.439419   0.356364    0.000416059
    0.566599    0.463222  0.0700088  0.805056   0.00117867  0.0320779  
    0.283351    0.332922  0.228347   0.616892   0.114793    0.0709104  
    0.00171797  0.734246  0.0182332  0.397453   0.103652    0.318561   
    0.14639     0.549599  0.676946   0.0075911  0.0615707   0.0354695  
    0.354003    0.824943  0.521569   0.0277972  0.154435    0.363959   